In [1]:
!pip install -U sagemaker

  Using cached sagemaker-2.251.1-py3-none-any.whl.metadata (17 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
Using cached sagemaker-2.251.1-py3-none-any.whl (1.7 MB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 238.7 MB/s eta 0:00:00
Using cached s3transfer-0.14.0-py3-none-any.whl (85 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Successfully uninstalled attrs-23.2.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.1
    Uninstalling botocore-1.37.1:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [botocore]
      Successfully uninstalled botocore-1.37.1━━━━━━━━━━━━━━━━ 1/5 [botocore]
  Attempting uninstall: s3transferm━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [botocore]
    Found existing installation: s3transfer 0.11.3━━━━━━━━━━━━ 1/5 [botocore]
    Un

In [13]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

In [14]:
region = 'us-east-1'

In [16]:
import boto3

boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)

In [17]:
s3_bucket = sagemaker_session.default_bucket()
s3_folder = "gpt2_fsdp_training"
dataset_folder = "datasets"
dataset_name = "wikitext"
resume = True

In [ ]:
secrets = boto3.client("secretsmanager")
wandb_api_key = secrets.get_secret_value(SecretId="wandb_api_key")["SecretString"]

In [18]:
from sagemaker.pytorch.estimator import PyTorch

estimator = PyTorch(
    entry_point='infra/sagemaker/distributed_train.sh',
    source_dir='gpt2_distributed_training/',
    role=role,
    instance_count=2,
    instance_type='ml.g5.xlarge',
    output_path=f's3://{s3_bucket}/{s3_folder}/sagemaker_output/',
    checkpoint_s3_uri=f"s3://{s3_bucket}/{s3_folder}/outputs",
    checkpoint_local_path="/opt/ml/checkpoints",
    dependencies=["gpt2_distributed_training/requirements.txt"],
    environment={
        'HYDRA_FULL_ERROR': '1',
        "WANDB_API_KEY": wandb_api_key,
    },
    hyperparameters={
        "trainer": "fsdp",
        "dataset.train_dataset.data_root": f"/opt/ml/input/data/{dataset_name}",
        "dataset.val_dataset.data_root": f"/opt/ml/input/data/{dataset_name}",
        "trainer.resume": resume,
    },
    py_version='py312',
    framework_version='2.7.1',
    # enable_sagemaker_metrics=True,
)

In [ ]:
estimator.fit({
    'wikitext': f's3://{s3_bucket}/{dataset_folder}/{dataset_name}'
})